# **Ejercicio 4 (modelo de regresión)**

In [20]:
import warnings
warnings.filterwarnings('ignore')
import mglearn
import matplotlib
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (mean_absolute_error, mean_squared_error, 
                             r2_score, mean_absolute_percentage_error)

In [3]:
detrain = pd.read_parquet(r'C:\Users\Usuario\Downloads\open-problems-single-cell-perturbations\de_train.parquet')

Obtenemos y asignamos nuestras variables de estudio.

In [45]:
celulas = pd.get_dummies(detrain, columns=['cell_type'])
cell_columns = [col for col in celulas.columns if 'cell_type' in col]
df_cells = celulas[cell_columns]
sm_namescolumn = pd.get_dummies(detrain['sm_name'], prefix='sm_name')
X = pd.concat([df_cells, sm_namescolumn], axis=1)
X = X.astype(int)


In [43]:
Y = detrain.iloc[:, 5:] 


In [44]:
Y

,A1BG,A1BG-AS1,A2M,A2M-AS1,A2MP1,A4GALT,AAAS,AACS,AAGAB,AAK1,...,ZUP1,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11B,ZYX,ZZEF1
0,0.104720,-0.077524,-1.625596,-0.144545,0.143555,0.073229,-0.016823,0.101717,-0.005153,1.043629,...,-0.227781,-0.010752,-0.023881,0.674536,-0.453068,0.005164,-0.094959,0.034127,0.221377,0.368755
1,0.915953,-0.884380,0.371834,-0.081677,-0.498266,0.203559,0.604656,0.498592,-0.317184,0.375550,...,-0.494985,-0.303419,0.304955,-0.333905,-0.315516,-0.369626,-0.095079,0.704780,1.096702,-0.869887
2,-0.387721,-0.305378,0.567777,0.303895,-0.022653,-0.480681,0.467144,-0.293205,-0.005098,0.214918,...,-0.119422,-0.033608,-0.153123,0.183597,-0.555678,-1.494789,-0.213550,0.415768,0.078439,-0.259365
3,0.232893,0.129029,0.336897,0.486946,0.767661,0.718590,-0.162145,0.157206,-3.654218,-0.212402,...,0.451679,0.704643,0.015468,-0.103868,0.865027,0.189114,0.224700,-0.048233,0.216139,-0.085024
4,4.290652,-0.063864,-0.017443,-0.541154,0.570982,2.022829,0.600011,1.231275,0.236739,0.338703,...,0.758474,0.510762,0.607401,-0.123059,0.214366,0.487838,-0.819775,0.112365,-0.122193,0.676629
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,-0.014372,-0.122464,-0.456366,-0.147894,-0.545382,-0.544709,0.282458,-0.431359,-0.364961,0.043123,...,-0.549987,-2.200925,0.359806,1.073983,0.356939,-0.029603,-0.528817,0.105138,0.491015,-0.979951
610,-0.455549,0.188181,0.595734,-0.100299,0.786192,0.090954,0.169523,0.428297,0.106553,0.435088,...,-1.236905,0.003854,-0.197569,-0.175307,0.101391,1.028394,0.034144,-0.231642,1.023994,-0.064760
611,0.338168,-0.109079,0.270182,-0.436586,-0.069476,-0.061539,0.002818,-0.027167,-0.383696,0.226289,...,0.077579,-1.101637,0.457201,0.535184,-0.198404,-0.005004,0.552810,-0.209077,0.389751,-0.337082
612,0.101138,-0.409724,-0.606292,-0.071300,-0.001789,-0.706087,-0.620919,-1.485381,0.059303,-0.032584,...,0.005951,-0.893093,-1.003029,-0.080367,-0.076604,0.024849,0.012862,-0.029684,0.005506,-1.733112


In [46]:
X

,cell_type_B cells,cell_type_Myeloid cells,cell_type_NK cells,cell_type_T cells CD4+,cell_type_T cells CD8+,cell_type_T regulatory cells,sm_name_5-(9-Isopropyl-8-methyl-2-morpholino-9H-purin-6-yl)pyrimidin-2-amine,sm_name_ABT-199 (GDC-0199),sm_name_ABT737,sm_name_AMD-070 (hydrochloride),...,sm_name_Tivozanib,sm_name_Topotecan,sm_name_Tosedostat,sm_name_Trametinib,sm_name_UNII-BXU45ZH6LI,sm_name_Vandetanib,sm_name_Vanoxerine,sm_name_Vardenafil,sm_name_Vorinostat,sm_name_YK 4-279
0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
610,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
611,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
612,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [50]:
X_train, X_test, y_train, y_test = train_test_split(X, Y,random_state=10)

## K-nn

### regresión

In [51]:
from sklearn.metrics import make_scorer, mean_absolute_percentage_error, mean_absolute_error, mean_squared_error, r2_score
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV

# Definir la función MRRMSE como un valor positivo
def mrrmse(y_true, y_pred):
    return np.sqrt(np.mean((y_true - y_pred)**2)) / np.mean(y_true)

# Hacer el scorer para MRRMSE
mrrmse_scorer = make_scorer(mrrmse, greater_is_better=True)  # Se usa greater_is_better=True porque queremos minimizar el error

# Definir el pipeline con escalado y modelo KNN
pipeline_knn = Pipeline([('scaler', MinMaxScaler()), ('knn', KNeighborsRegressor())])

# Definir los hiperparámetros para GridSearchCV
param_grid_knn = {'knn__n_neighbors': [1,10,100,1000,10000]}  # Ajusta los valores según sea necesario

# Definir el GridSearchCV para KNN (usando MRRMSE como métrica)
grid_search_knn = GridSearchCV(estimator=pipeline_knn, param_grid=param_grid_knn, cv=5, scoring=mrrmse_scorer)

# Entrenar el GridSearchCV con los datos de entrenamiento
grid_search_knn.fit(X_train, y_train)

# Obtener los mejores parámetros y el mejor score
best_parameters_knn = grid_search_knn.best_params_
best_score_knn = grid_search_knn.best_score_

print("Mejores parámetros KNN: ", best_parameters_knn)
print("Mejor MRRMSE score KNN: ", best_score_knn)

# Predecir con el modelo KNN
model_pred_knn = grid_search_knn.predict(X_test)

# Crear tabla con las métricas de error para KNN
tabla_knn = {
    '': ['K-NN'],
    'MAPE': [mean_absolute_percentage_error(y_test, model_pred_knn)],
    'MAE': [mean_absolute_error(y_test, model_pred_knn)],
    'RMSE': [np.sqrt(mean_squared_error(y_test, model_pred_knn))],
    'MSE': [mean_squared_error(y_test, model_pred_knn)],
    'R2': [r2_score(y_test, model_pred_knn)]
}

# Convertir a DataFrame para mejor visualización
tabla_knn_df = pd.DataFrame(tabla_knn)

print(tabla_knn_df)


Mejores parámetros KNN:  {'knn__n_neighbors': 1}
Mejor MRRMSE score KNN:  8.14886612415
              MAPE       MAE      RMSE       MSE        R2
0  K-NN  15.085849  0.969467  2.276153  5.180871 -0.388531


Luego de efectuar el modelo, nos dió que el mejor score con la métrica MRRMSE fue de **8.1488661** para k=1.


## Regresión lineal

In [65]:
def MRRMSE(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred)) / np.mean(y_true)

pipeline_lr = Pipeline([
    ('scaler', StandardScaler()),
    ('lr', LinearRegression())
])

pipeline_lr.fit(X_train, y_train)
model_pred_lr = pipeline_lr.predict(X_test)

mrrmse_score_lr = MRRMSE(y_test, model_pred_lr)

tabla_lr = {
    '': ['Regresión Lineal'],
    'MAPE': [mean_absolute_percentage_error(y_test, model_pred_lr)],
    'MAE': [mean_absolute_error(y_test, model_pred_lr)],
    'RMSE': [np.sqrt(mean_squared_error(y_test, model_pred_lr))],
    'MSE': [mean_squared_error(y_test, model_pred_lr)],
    'R2': [r2_score(y_test, model_pred_lr)]
}

tabla_lr_df = pd.DataFrame(tabla_lr)

print(f'Mejor MRRMSE score LR: {mrrmse_score_lr:.10f}')
print(tabla_lr_df.to_string(index=False))


Mejor MRRMSE score LR: 6.0331719475
                      MAPE      MAE     RMSE     MSE        R2
Regresión Lineal 15.453345 1.077885 2.708289 7.33483 -0.144614


Por otro lado, para el modelo de regresión lineal obtuvimos que el mejor score es de **6.03317**.

$$
\begin{array}{|c|c|c|c|c|c|}
\hline
\text{Modelo} & \text{MAPE} & \text{MAE} & \text{RMSE} & \text{MSE} & R^2 \\
\hline
\text{K-NN} &15.085849 & 0.969467 & 2.276153&5.180871 & -0.388531\\
\hline
\text{Regresión Lineal} & 15.453345 &1.077885 &2.708289 &7.33483 & -0.144614\\
\hline
\end{array}
$$


**Conclusiones**: La regresión lineal muestra un mejor rendimiento en términos de MRRMSE, lo que indica su eficacia en este contexto. Sin embargo, es importante resaltar que ambos modelos presentan limitaciones en su capacidad para explicar la variabilidad de los datos, posiblemente debido a la presencia de valores atípicos o características específicas del conjunto de datos. Por lo tanto, se recomienda explorar otros modelos que puedan ofrecer mejores predicciones para nuestras variables de interés.